In [2]:
!pip install torch  --quiet

# # Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

# #FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
!pip install -U transformers
# # !pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# # Uncomment only if you're using A100 GPU
# #!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet

# %pip install -U wandb

In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset, DatasetDict
from trl import SFTTrainer, setup_chat_format

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')

In [6]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF_TOKEN")

login(token = hf_token)

wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama 3 8B on SQL dataset', 
    job_type="training", 
    anonymous="allow"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: huyhoangt2201 (huyhoangt2201-fpt-university). Use `wandb login --relogin` to force relogin


In [7]:
base_model = "phamhai/Llama-3.2-1B-Instruct-Frog"
new_model = "llama-3.2-1b-sql_finetuned_multitableJidouka_1.0"

In [8]:
torch_dtype = torch.float16

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float32
    #attn_implementation=attn_implementation
)
tokenizer = AutoTokenizer.from_pretrained(base_model,use_fast=True)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [9]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [10]:
dataset_train = load_dataset("huyhoangt2201/multitableJidouka-1.1-english-only-dataset", split='train[:90%]')
dataset_val = load_dataset("huyhoangt2201/multitableJidouka-1.1-english-only-dataset", split='train[-10%:]')
dataset = DatasetDict({
    'train': dataset_train,
    'validation': dataset_val
})
dataset.save_to_disk("completed_train_dataset")

(…)07_records_english_multitableJidouka.csv:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/507 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/456 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/51 [00:00<?, ? examples/s]

In [ ]:
dataset_train2 = load_dataset("huyhoangt2201/jidouka3.1", split='train[:90%]')
dataset_val2 = load_dataset("huyhoangt2201/jidouka3.1", split='train[-10%:]')
dataset2 = DatasetDict({
    'train': dataset_train2,
    'validation': dataset_val2
})

In [ ]:
def format_context(sample):
    sample['context'] = prompt_template
    return sample

In [ ]:
dataset_train2_2 = dataset_train2.map(format_context, batched=False)
dataset_val2_2 = dataset_val2.map(format_context, batched=False)

In [ ]:
dataset_train2_2['question'][1]

In [11]:
dataset['train'][0]['context']

" \nYou are an SQL query assistant. Based on schema below, generate an SQL query to retrieve the relevant information for the user. If the user’s question is unrelated to the table, respond naturally in user's language.\n\nSchema:\n-- Table: Job\nCREATE TABLE Job (\n    Id INT PRIMARY KEY AUTO_INCREMENT,\n    Job_name NVARCHAR(255) NOT NULL,\n    DateCreate DATETIME NOT NULL,\n    ModifiedDate DATETIME,\n    CreateBy NVARCHAR(255),\n    ModifiedBy NVARCHAR(255)\n);\n\n-- Table: Department\nCREATE TABLE Department (\n    Id INT PRIMARY KEY AUTO_INCREMENT,\n    Department_name NVARCHAR(255) NOT NULL,\n    DateCreate DATETIME NOT NULL,\n    ModifiedDate DATETIME,\n    CreateBy NVARCHAR(255),\n    ModifiedBy NVARCHAR(255)\n);\n\n-- Table: Author\nCREATE TABLE Author (\n    Id INT PRIMARY KEY AUTO_INCREMENT,\n    Author_name NVARCHAR(255) NOT NULL,\n    DateCreate DATETIME NOT NULL,\n    ModifiedDate DATETIME,\n    CreateBy NVARCHAR(255),\n    ModifiedBy NVARCHAR(255)\n);\n\n-- Table: Tool\

In [12]:
def format_data_template(sample):
    chat = [
          {"role":"system", "content": sample['context']},
          {"role":"user", "content":sample['question']},
          {"role":"assistant","content":sample['answer']}
    ]
    return {
        "messages": tokenizer.apply_chat_template(chat, tokenize=False)
    }

In [ ]:
def format_data_template_to_token(sample):
    chat = [
          {"role":"system", "content": sample['context']},
          {"role":"user", "content":sample['question']}
    ]
    sample['input_ids'] = tokenizer.apply_chat_template(chat, tokenize=True, padding=True, truncation=True, return_tensors='pt')
    sample['labels'] = tokenizer(sample['answer'], padding=True, truncation=True, return_tensors='pt').input_ids
    
    return sample
tokenized_dataset_train = dataset['train'].map(format_data_template_to_token, remove_columns=['context','question','answer'], batched=True)
tokenized_dataset_valid = dataset['validation'].map(format_data_template_to_token, remove_columns=['context','question', 'answer'], batched=True)

In [13]:
dataset_train = dataset['train'].map(format_data_template, remove_columns=['context','question','answer'])

Map:   0%|          | 0/456 [00:00<?, ? examples/s]

In [14]:
dataset_valid = dataset['validation'].map(format_data_template, remove_columns=['context', 'question','answer'])

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [15]:
dataset_train['messages'][0]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 19 Nov 2024\n\nYou are an SQL query assistant. Based on schema below, generate an SQL query to retrieve the relevant information for the user. If the user’s question is unrelated to the table, respond naturally in user's language.\n\nSchema:\n-- Table: Job\nCREATE TABLE Job (\n    Id INT PRIMARY KEY AUTO_INCREMENT,\n    Job_name NVARCHAR(255) NOT NULL,\n    DateCreate DATETIME NOT NULL,\n    ModifiedDate DATETIME,\n    CreateBy NVARCHAR(255),\n    ModifiedBy NVARCHAR(255)\n);\n\n-- Table: Department\nCREATE TABLE Department (\n    Id INT PRIMARY KEY AUTO_INCREMENT,\n    Department_name NVARCHAR(255) NOT NULL,\n    DateCreate DATETIME NOT NULL,\n    ModifiedDate DATETIME,\n    CreateBy NVARCHAR(255),\n    ModifiedBy NVARCHAR(255)\n);\n\n-- Table: Author\nCREATE TABLE Author (\n    Id INT PRIMARY KEY AUTO_INCREMENT,\n    Author_name NVARCHAR(255) NOT NULL,\n    DateCreate DA

In [16]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=10,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

In [17]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    peft_config=peft_config,
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
    dataset_text_field='messages',
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/456 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [18]:
eot = "<|eot_id|>"
eot_id = tokenizer.convert_tokens_to_ids(eot)
tokenizer.pad_token = eot
tokenizer.pad_token_id = eot_id

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
456,0.002700,0.002733
912,0.002600,0.002721
1368,0.003300,0.002718
1824,0.002700,0.002717


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


In [ ]:
wandb.finish()
model.config.use_cache = True

In [ ]:
new_model = 'llama-3.2-1b-sql_finetuned_multitableJidouka_1.0_adapter'
new_model

In [ ]:
trainer.model.save_pretrained(new_model)
trainer.model.push_to_hub(new_model, use_temp_dir=False)

In [ ]:
new_model = 'huyhoangt2201/llama-3.2-1b-sql_finetuned_multitableJidouka_1.0_adapter
base_model = 'phamhai/Llama-3.2-1B-Instruct-Frog'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

# base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

# Merge adapter with base model
merge_model = PeftModel.from_pretrained(base_model_reload, new_model)

merge_model = merge_model.merge_and_unload()

In [ ]:
new_model_merged = 'llama-3.2-1b-sql_finetuned_mutitableJidouka_1.0_merged'

In [ ]:
merge_model.save_pretrained(new_model_merged)
tokenizer.save_pretrained(new_model_merged)

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF_TOKEN")
login(token = hf_token)

In [ ]:
merge_model.push_to_hub(new_model_merged, use_temp_dir=False)
tokenizer.push_to_hub(new_model_merged, use_temp_dir=False)

## new_model inference

In [ ]:
new_model_name = 'huyhoangt2201/llama-3.2-1b-sql_finetuned_billingual_2.0_merged'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
device = torch.device('cuda')
model_path = new_model_name
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to('cuda')

In [ ]:
prompt_template = """
You are an SQL query assistant. Based on the table information below, generate an SQL query to retrieve the relevant information for the user. If the user’s question is unrelated to the table, respond naturally in human language.

The jidouka table contains the following columns:
id: Row identifier (int)
tên_cải_tiến: Name of the improvement (str)
loại_hình_công_việc: Type of work that the improvement is intended to enhance (str) (e.g., database processing, data entry, workflow optimization, etc.)
công_cụ: Tool used to achieve the improvement (str) (e.g., Python, Excel, Visual Studio Code, etc.)
mô_tả: Detailed description of the improvement (str) (e.g., each step of the improvement process)
sản_phẩm: Output product of the improvement (str) (e.g., .csv file, .xlsx file, etc.)
tác_giả: Contributor, company employee, or creator of the improvement (str)
bộ_phận: Department of the author, usually referred to as "dc" (str) (e.g., dc1, dc2, dc3, dcd, souko, etc.)
số_giờ: Number of hours saved by applying the improvement (int)
số_công_việc_áp_dụng: Number of tasks in the company that the improvement has supported (int)
thời_điểm_ra_mắt: Launch date of the tool (str) (e.g., 2024-10-11, 2024-10-09, etc.)
thông_tin_thêm: Link to additional documentation (PowerPoint, video) on using the improvement or the improvement’s tool (str)
"""

In [ ]:
from typing import List, Dict
class ContextAwareChatbot:
    def __init__(self,prompt, max_history: int = 5):
        self.model = model
        self.tokenizer = tokenizer 
        self.max_history = max_history
        self.conversation_history: List[Dict[str, str]] = []
        self.prompt=prompt
    def _build_prompt(self) -> str:
        # Build context from history

        return self.prompt

    def _clean_response(self, response: str) -> str:
        # Clean up the generated response
        response = response.split("Assistant:")[-1].strip()
        # Stop at any new "Human:" or "Assistant:" markers
        if "Human:" in response:
            response = response.split("Human:")[0].strip()
        return response

    def chat(self, user_input: str) -> str:
        # Generate the contextualized prompt
        prompt = self._build_prompt()

#         # Generate response
#         response = self.pipeline(
#             prompt,
#             return_full_text=False,
#             clean_up_tokenization_spaces=True
#         )[0]['generated_text']

#         # Clean the response
#         cleaned_response = self._clean_response(response)
        messages =[
            {'role':'system',
             'content':prompt}
            ,
            {'role':'user',
             'content':user_input}
        ]
        tokenized_chat = self.tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors='pt').to('cuda')
        outputs = self.model.generate(tokenized_chat, max_new_tokens=256).to('cuda')
        bot_response = self.tokenizer.decode(outputs[0])
        bot_response = bot_response.split('<|start_header_id|>assistant<|end_header_id|>')
        bot_response = bot_response[1].strip()[:-10]
        # Update conversation history
        self.conversation_history.append({
            'human': user_input,
            'assistant': bot_response
        })

        return bot_response

    def get_history(self) -> List[Dict[str, str]]:
        return self.conversation_history

    def clear_history(self):
        self.conversation_history = []

# 4. Create chatbot instance
chatbot = ContextAwareChatbot(prompt_template)

# 5. Example usage function
def chat_session():
    print("Chatbot initialized. Type 'exit' to end the conversation, 'clear' to clear history.")

    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
        elif user_input.lower() == 'clear':
            chatbot.clear_history()
            print("Conversation history cleared!")
            continue

        response = chatbot.chat(user_input)
        print(f"\nAssistant: {response}")

# 6. Example of how to use
if __name__ == "__main__":
    chat_session()